In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Authorship

A notebook that explores authorship.

## Notebook setup

Ensure that necessary libraries are install and imported into the workplace.

In [2]:
# setup relative path to import local module (needed when used in Conda) https://stackoverflow.com/questions/34478398
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import visualising_poetry.data as vpd
import visualising_poetry.widgets as vpw
import visualising_poetry.plot as vpp

# import libraries
import ipywidgets as widgets
from IPython.display import display, HTML

# get data and process (if necessary)
vpd.setup_if_needed()

# get data as a single data frame
df = vpd.complete_dataset()

## Authorship and number of poems

The following table shows authors who have been attributed to the authorship of twenty or more poems

In [3]:
authors = vpd.authorship_overview_df(df)
authors_20 = authors[authors['Total Poems'] >= 20]
display(HTML(authors_20.to_html()))

## Numbers of authors attributed to poems

The following scatter plot breaks down the number of authors attributed to poems. We can see
the majority are only attributed to a single poem, with a long tail of authors attributed
to multiple poems.

In [4]:
authors_poems = vpd.authorship_poems_overview_df(df)
vpp.author_poems_plot(authors_poems)

## Authors and publications

The following heat map shows the number of times authors were printed in publications.
The authors has been limited to those who have been published 20 or more times.

In [5]:
author_pubs = vpd.authorship_publications_overview_df(df)
vpp.authorship_publications_overview_plot(author_pubs)

## Authors and years of publication

The following heat map shows the number of times authors were printed in publications.
The authors has been limited to those who have been published 20 or more times.

In [6]:
author_pub_years = vpd.authorship_publication_year_overview_df(df)
vpp.authorship_publication_year_plot(author_pub_years)

## Individual author

In [7]:
author_list = vpd.unique_author_list(df, pub_no=2)
author_widget = vpw.author_list_widget(df)
btn = widgets.Button(description='Generate Report')
# output for our widget
out = widgets.Output(layout={'border': '1px solid black'})

def button_pressed(button):
    out.clear_output()
    author = author_widget.value
    
    results_df = vpd.authors_expanded(df, [author])
    results_summary = results_df[vpd.SUMMARY]
    with out:
        display(HTML('<h2>Report for \'{}\'</h2>'.format(author)))
        display(HTML('<h3>Summary of data for \'{}\'</h3>'.format(author)))
        display(HTML(results_summary.to_html()))
        matrix = vpd.author_publications(df, author)
        display(HTML('<h3>Heat map of publication and year for {}'.format(author)))
        vpp.authorship_publication_year_plot(matrix)
        unique_vs_copies = vpd.author_unique_vs_copies(df, author)
        display(HTML('<h3>Unique versus copies for  \'{}\'</h3>'.format(author)))
        display(HTML(unique_vs_copies.to_html()))
        unique_vs_copies2 = unique_vs_copies[['No. of poems appearing once', 'No. of copies']]
        unique_vs_copies2 = unique_vs_copies2.transpose()
        unique_vs_copies2.T.plot.bar(stacked=True)
        
        display(HTML('<h3>Heat map of poems and years for  \'{}\'</h3>'.format(author)))
        poems_years = vpd.author_poem_matrix(df, author)
        vpp.heat_map_plot(poems_years, title=author)

btn.on_click(button_pressed)
display(author_widget)
display(btn)
out